In [ ]:
import numpy as np
import skimage.io as io
import json
from PIL import Image, ImageDraw, ImageFont
import os
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('/home/dan/work/cocoapi/PythonAPI/')

from pycocotools.coco import COCO

In [ ]:
DATA_DIR = '/home/gpu2/hdd/dan/COCO'
DATA_TYPE = 'val2017'  # train2017 or val2017
RESULT_PATH = '/home/gpu2/hdd/dan/COCO/val_annotations/'

In [ ]:
coco = COCO('{}/annotations/instances_{}.json'.format(DATA_DIR, DATA_TYPE))

# Show categories

In [ ]:
categories = coco.loadCats(coco.getCatIds())
coco_id_to_name = {c['id']: c['name'] for c in categories}
names = [cat['name'] for cat in categories]
print('COCO categories: \n{}'.format(' '.join(names)))
print('\nnumber of labels:', len(names))

# Create label encoding

In [ ]:
# number of row - integer encoding
with open('data/labels.txt', 'w') as f:
    for n in names:
        f.write(n + '\n')

# Show an image with full annotation

In [ ]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person', 'zebra'])
imgIds = coco.getImgIds(catIds=catIds)
image = coco.loadImgs(imgIds[np.random.randint(0, len(imgIds))])[0]

In [ ]:
# load and display image and instance annotations
I = io.imread('%s/images/%s/%s' % (DATA_DIR, DATA_TYPE, image['file_name']))
annIds = coco.getAnnIds(imgIds=image['id'], catIds=catIds)
annotations = coco.loadAnns(annIds)

plt.imshow(I)
plt.axis('off')
coco.showAnns(annotations)

# Show bounding box annotation

In [ ]:
def draw_boxes_on_image(image, annotations):

    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image, 'RGBA')
    width, height = image.size

    for a in annotations:
        xmin, ymin, w, h = a['bbox']
        xmax, ymax = xmin + w, ymin + h

        fill = (255, 255, 255, 45)
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
    return image

In [ ]:
draw_boxes_on_image(I, annotations)

# Convert

In [ ]:
imgIds = coco.getImgIds()
print('number of images:', len(imgIds))

In [ ]:
def get_annotation(i):
    image = coco.loadImgs(i)[0]
    annIds = coco.getAnnIds(imgIds=image['id'])
    annotation = {
      "filename": image['file_name'],
      "size": {"depth": 3, "width": image['width'], "height": image['height']}
    }
    objects = []
    for a in coco.loadAnns(annIds):
        label = coco_id_to_name[a['category_id']]
        xmin, ymin, w, h = a['bbox']
        xmax, ymax = xmin + w, ymin + h
        objects.append({"bndbox": {"ymin": ymin, "ymax": ymax, "xmax": xmax, "xmin": xmin}, "name": label})
    annotation["object"] = objects
    return annotation

In [ ]:
shutil.rmtree(RESULT_PATH, ignore_errors=True)
os.mkdir(RESULT_PATH)

In [ ]:
for i in tqdm(imgIds):
    d = get_annotation(i)
    filename = d['filename']
    assert filename.endswith('.jpg')
    name = filename[:-4]
    json.dump(d, open(os.path.join(RESULT_PATH, name + '.json'), 'w')) 